In [24]:
import jsonrpclib
from simplejson import loads
from nltk import sent_tokenize,word_tokenize

In [14]:
server = jsonrpclib.Server("http://localhost:8080")

In [15]:
string = "Barack Obama is the president of the USA. He will leave the office this year."
result = loads(server.parse(string))

In [16]:
print string
print result['coref']

Barack Obama is the president of the USA. He will leave the office this year.
[[[[u'the president of the USA', 0, 4, 3, 8], [u'Barack Obama', 0, 1, 0, 2]], [[u'He', 1, 0, 0, 1], [u'Barack Obama', 0, 1, 0, 2]]]]


In [23]:
import wikipedia as wk
batman = wk.page('Batman')
print type(batman.content)

<type 'unicode'>


In [53]:
sents = sent_tokenize(batman.content)

In [54]:
string = ''
for i in sents[0:10]:
    string += i
print string

Batman is a fictional superhero appearing in American comic books published by DC Comics.The character was created by artist Bob Kane and writer Bill Finger, and first appeared in Detective Comics #27 (May 1939).Originally named the "Bat-Man", the character is also referred to by such epithets as the Caped Crusader, the Dark Knight, and the World's Greatest Detective.Batman's secret identity is Bruce Wayne, an American billionaire, playboy, philanthropist, and owner of Wayne Enterprises.After witnessing the murder of his parents Thomas Wayne and Martha Wayne as a child, he swore revenge on criminals, an oath tempered by a sense of justice.Wayne trains himself physically and intellectually and crafts a bat-inspired persona to fight crime.Batman operates in the fictional Gotham City, with assistance from various supporting characters, including his butler Alfred, police commissioner Jim Gordon, and vigilante allies such as Robin.Unlike most superheroes, Batman does not possess any superp

In [55]:
result = loads(server.parse(string))
coreferences = result['coref']

In [61]:
for chunk in coreferences:
    print "***********************************"
    for text in chunk:
        print '{} is referenced to {}'.format(str(text[0][0]),str(text[1][0]))

***********************************
Batman is referenced to Batman 's
a fictional superhero appearing in American comic books published by DC Comics.The character is referenced to Batman 's
Batman is referenced to Batman 's
his is referenced to Batman 's
Batman is referenced to Batman 's
***********************************
the character is referenced to DC Comics.The character
***********************************
he is referenced to the `` Bat-Man ''
his is referenced to the `` Bat-Man ''
his is referenced to the `` Bat-Man ''
his is referenced to the `` Bat-Man ''
***********************************
the Caped Crusader , the Dark Knight , and the World 's Greatest Detective.Batman 's secret identity is referenced to Martha Wayne as a child
Bruce Wayne , an American billionaire , playboy , philanthropist , and owner of Wayne Enterprises.After witnessing the murder of his parents Thomas Wayne and Martha Wayne as a child , he swore revenge on criminals , an oath tempered by a sense of just